In [28]:
import plotly
import cufflinks
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import numpy as np
import cvxopt as opt
from cvxopt import blas, solvers
import yfinance as yf
import matplotlib.pyplot as plt

In [160]:
prices = yf.download(["PETR4.SA", 'BBDC4.SA', 'TASA4.SA'])['Adj Close'].fillna(0)

[*********************100%***********************]  3 of 3 completed


In [138]:
r = prices.pct_change()
solvers.options['show_progress'] = False

In [123]:
w = np.array([0.2, 0.2, 0.2])
R = np.dot(r, w) #retorno do portfólio

In [135]:
mean_returns = r.mean()

In [136]:
mean_returns

BBDC4.SA         inf
PETR4.SA    0.000795
TASA4.SA    0.000992
dtype: float64

In [7]:
covariance = np.cov(r[1:].T)

In [8]:
vol = np.sqrt(np.dot(w.T, np.dot(covariance, w)))

In [161]:
def generate_wallets(prices, num_portfolios = 5000, risk_free = 0.000358):
    # vetores de dados
    portfolio_weights = []
    portfolio_exp_returns = []
    portfolio_vol = []
    portfolio_sharpe = []

    # retorno simples 
    r = np.log(prices).diff().fillna(0)
    r.replace([np.inf, -np.inf], 0, inplace=True)
    mean_returns = r.mean()

    # matriz de covariância 
    covariance = np.cov(r[1:].T)

    for i in range(num_portfolios):
        # gerando pesos aleatórios
        k = np.random.rand(len(r.columns))
        w = k / sum (k)

        # retorno
        R = np.dot(mean_returns, w) # produto matricial

        # risco
        vol = np.sqrt(np.dot(w.T, np.dot(covariance, w)))

        # sharpe ratio
        sharpe = (R - risk_free)/vol

        portfolio_weights.append(w)
        portfolio_exp_returns.append(R)
        portfolio_vol.append(vol)
        portfolio_sharpe.append(sharpe)

    wallets = {'weights': portfolio_weights,
              'returns': portfolio_exp_returns,
              'vol':portfolio_vol,
              'sharpe': portfolio_sharpe}

    return wallets

In [162]:
def best_portfolio(wallets, method = 'sharpe_ratio'):
    vol = wallets['vol']
    sharpe = wallets['sharpe']
    weights = wallets['weights']
    returns = wallets['returns']
    
    if method == 'sharpe_ratio':

        indice = np.array(sharpe).argmax()

    elif method == 'volatility':

        indice = np.array(vol).argmin()

    elif method == 'return':

        indice = np.array(returns).argmax()

    return weights[indice]

In [163]:
def plot_efficient_frontier(wallets, method = 'sharpe_ratio'):
    vol = wallets['vol']
    returns = wallets['returns']
    sharpe = wallets['sharpe']

    if method == 'sharpe_ratio':

        indice = np.array(sharpe).argmax()
        y_axis = returns[indice]
        X_axis = vol[indice]

    elif method == 'volatility':

        indice = np.array(vol).argmin()
        y_axis = returns[indice]
        X_axis = vol[indice]

    elif method == 'return': 

        indice = np.array(returns).argmax()
        y_axis = returns[indice]
        X_axis = vol[indice]

#    plt.scatter(vol, returns, c = sharpe, cmap = 'viridis')
#    plt.scatter(X_axis, y_axis, c = 'red', s = 50)
#    plt.title("Efficient Frontier")
#    plt.xlabel("Volatility")
#    plt.ylabel("Expected return")
#    plt.show()

    return X_axis, y_axis

In [ ]:
wallets = generate_wallets(prices)
best_portfolio = best_portfolio(wallets)
x,y = plot_efficient_frontier(wallets)

In [166]:
data = [go.Scatter(
            x=wallets['vol'],
            y=wallets['returns'],
            mode='markers',            
             marker=dict(
                    color=wallets['sharpe'],
                    showscale=False,
                    cmax= max(wallets['sharpe'], key= int),
                    cmin=0,                 
                    )       
        )
       ]

title_text = "Média e desvio padrão dos retornos para os portfólios gerados"
title={'text': title_text, 'xanchor': 'center', 'yanchor': 'bottom', 'y':0, 'x':0.5,}

fig = go.Figure(data=data)

fig.update_layout(title=title, 
            xaxis_rangeslider_visible=False,  width=1280, height=720,
            xaxis_showgrid=True, xaxis_gridwidth=1, xaxis_gridcolor='#E8E8E8',
            yaxis_showgrid=True, yaxis_gridwidth=1, yaxis_gridcolor='#E8E8E8',
            yaxis_title="Média",
            xaxis_title="Desvio padrão",                  
            plot_bgcolor='rgba(0,0,0,0)'              
                 )
                  
fig.show()